In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

## File Prep ##

In [ ]:
%%time
column_names=['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount', 'Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount', 'FavoriteCount', 'Text']
tweets = pd.read_csv('clinton_trump_tweets.txt', sep="\t",  encoding="ISO-8859-1", header=None, names=column_names)
#print initial data
tweets.drop(['ScreenName', 'FollowersCount', 'FriendsCount', 'CreatedAt', 'StatusID'], axis=1)
tweets.Location = tweets.Location.astype(str)
tweets.Text = tweets.Text.astype(str)

### Task 1.1 (10 pts): Remove all retweets first. Remove all users that have less than 20 tweets. You may want to keep the entire tweet content, including hashtags/handles. ###

In [ ]:
tweets = tweets[~tweets.Text.str.startswith('RT')]

In [ ]:
%%time
def keepHashMentions(text):
    hashMentions = []
    for word in text:
        hashMentions.extend(word)
    return " ".join(hashMentions).strip()
## keeping tweet including hashtags and mentions here ## 
tweets['HashMentions'] = tweets.Text.str.findall('(@\w+)|(#\w+)').apply(keepHashMentions)


### Remove all users that have less than 20 tweets. ###

In [ ]:
#1.2 keep tweets where UID appears 20+ times
tweets = tweets.groupby("UserID").filter(lambda x: len(x) >= 20)

In [ ]:
%%time
# Generate a list of mention/hashes that have a frequency of 20+ #
top_hash = pd.Series(tweets['HashMentions'].str.cat(sep=' ').split()).value_counts()
top20 = top_hash[top_hash>=20]
top20List = top20.index.tolist()

# Convert list to set so it will have quick lookup
top20Set = set(top20List)


# generate list of mention/hashes that occur 20+ times from our orginal list 
def removeUnder20Mentions(hashMentions):
    mentions =  hashMentions.split()
    mentionsOver20 = []
    for mention in mentions:
        if(mention in top20Set):
            mentionsOver20.append(mention)
    return " ".join(mentionsOver20)
tweets['FrequencyOver20'] = tweets.HashMentions.apply(removeUnder20Mentions)
tweets.head(20)


## Apply ground truths column to UserID of tweets Dataframe ##

In [ ]:
ground_truth = pd.read_table('clinton_trump_user_classes.txt', encoding ="ISO-8859-1", dtype=int, names=["UserID", "TrumpOrClinton"])
#ground_truth.UserID = ground_truth.UserID.astype(int)
mergeGroundTruths = pd.merge(tweets, ground_truth, on = 'UserID')
#ground truth

In [ ]:
mergeGroundTruths.head()

### 1.1: Use train_test_split() to split data into training and test sets, where 20 percent of the records go to test set. ###

In [ ]:
# dummyDataFrame = pd.DataFrame({'top20List': top20List})
tweets.HashMentions.str.get_dummies(sep=" ")
# pd.get_dummies(dummyDataFrame,prefix=['top20List'], drop_first=True)
# dummyDataFrame

In [ ]:
# split X and y into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False, random_state=5)

In [ ]:
len(tweets.Location.unique().tolist())

In [ ]:
len(tweets.Description.unique().tolist())